In [23]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Load data
stores = pd.read_csv('stores.csv')
train = pd.read_csv('train.csv')
features = pd.read_csv('features.csv')

In [24]:
# Merge datasets
data = pd.merge(train, stores, how='left', on='Store')
data = pd.merge(data, features, how='left', on=['Store', 'Date'])

# Data preprocessing
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values(by='Date')
data = data.fillna(0)  # Fill missing values with 0 for simplicity


In [25]:
# Feature engineering
# Add lag features for weekly sales
data['Lag_1_Weekly_Sales'] = data.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1)
data['Lag_2_Weekly_Sales'] = data.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(2)


In [16]:
# Normalization
scaler = MinMaxScaler()
data[['Weekly_Sales', 'Lag_1_Weekly_Sales', 'Lag_2_Weekly_Sales', 'Temperature', 'Fuel_Price',
      'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment']] = \
    scaler.fit_transform(data[['Weekly_Sales', 'Lag_1_Weekly_Sales', 'Lag_2_Weekly_Sales',
                               'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
                               'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment']])



In [26]:
# Sequence preparation
X = data[['Lag_1_Weekly_Sales', 'Lag_2_Weekly_Sales', 'Temperature', 'Fuel_Price',
           'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment']].values
y = data['Weekly_Sales'].values

X = X.reshape(X.shape[0], 1, X.shape[1])


In [27]:
# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')


In [28]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/50
10540/10540 [==============================] - 36s 3ms/step - loss: nan - val_loss: nan
Epoch 2/50
10540/10540 [==============================] - 33s 3ms/step - loss: nan - val_loss: nan
Epoch 3/50
10540/10540 [==============================] - 32s 3ms/step - loss: nan - val_loss: nan
Epoch 4/50
10540/10540 [==============================] - 32s 3ms/step - loss: nan - val_loss: nan
Epoch 5/50
10540/10540 [==============================] - 32s 3ms/step - loss: nan - val_loss: nan
Epoch 6/50
10540/10540 [==============================] - 32s 3ms/step - loss: nan - val_loss: nan
Epoch 7/50
10540/10540 [==============================] - 32s 3ms/step - loss: nan - val_loss: nan
Epoch 8/50
10540/10540 [==============================] - 32s 3ms/step - loss: nan - val_loss: nan
Epoch 9/50
10540/10540 [==============================] - 34s 3ms/step - loss: nan - val_loss: nan
Epoch 10/50
10540/10540 [==============================] - 33s 3ms/step - loss: nan - val_loss: nan
Epoch 11/

In [44]:
# ... (previous code)

# Load the test data
test_data = pd.read_csv('test.csv')
# Make predictions on the test dataset
test_data = pd.merge(test_data, stores, how='left', on='Store')
test_data = pd.merge(test_data, features, how='left', on=['Store', 'Date'])
test_data['Date'] = pd.to_datetime(test_data['Date'])
test_data = test_data.sort_values(by='Date')
test_data = test_data.fillna(0)

test_data['Lag_1_Weekly_Sales'] = test_data.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1)
test_data['Lag_2_Weekly_Sales'] = test_data.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(2)

test_data[['Lag_1_Weekly_Sales', 'Lag_2_Weekly_Sales', 'Temperature', 'Fuel_Price',
           'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment']] = \
    scaler.transform(test_data[['Weekly_Sales', 'Lag_1_Weekly_Sales', 'Lag_2_Weekly_Sales',
                                'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
                                'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment']])

X_test = test_data[['Lag_1_Weekly_Sales', 'Lag_2_Weekly_Sales', 'Temperature', 'Fuel_Price',
                    'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment']].values.reshape(
    test_data.shape[0], 1, 11
)



KeyError: 'Column not found: Weekly_Sales'

In [43]:
predictions = model.predict(X_test)

# Reshape predictions and apply inverse_transform
predictions = predictions.reshape(-1, 1)
test_data['Predicted_Weekly_Sales'] = scaler.inverse_transform(np.hstack([predictions, np.zeros_like(predictions)]))[:, 0]

# Display the predictions
print(test_data[['Store', 'Dept', 'Date', 'Predicted_Weekly_Sales']])


3596/3596 [==============================] - 7s 2ms/step


ValueError: operands could not be broadcast together with shapes (115064,2) (11,) (115064,2) 